**Outline**

The purpose of this script is to isolate "long" button press events (occurring at least 1 s after and 15 s before adjacent button presses) in the raw MEG data, perform preprocessing and epoching, and then compute a tfr for each trial.


**Import packages**

In [16]:
import os
import mne
import pandas as pd
import numpy as np
from tqdm import tqdm

from mne.preprocessing import ICA, create_ecg_epochs, create_eog_epochs
from statistics import mode

# Suppress output from mne unless it requires attention
mne.set_log_level('WARNING')

**Define data directories and filenames**

In [17]:
# Path to raw data
raw_path = os.path.join("/media/WDEasyStore/timb/camcan/download/20170824/cc700/meg/pipeline/release004/data_movecomp/aamod_meg_maxfilt_00002")

# Data path (from which we will read some files and write output)
data_path = os.path.join("/media/NAS/lbailey/PMBR_timecourse/output")

# Path to the event numpy files
event_numpy_path = os.path.join(data_path, "event_numpys")

# Path to Tim's old taskSensorAnalysis folder (containing ICA files and demographic csv file)
timb_task_sensor_path = os.path.join("/home/timb/camcan/proc_data/")

# Define generic filenames
raw_fif_fname = 'transdef_transrest_mf2pt2_task_raw.fif'
ica_fname = 'transdef_transrest_mf2pt2_task_raw-ica.fif'

**Import subject list and BP trial timing information**

In [18]:
# Load in the trial timing and demographics data.
df_trial_timings_allsubjects = pd.read_csv(os.path.join(data_path, "trial_timings.csv")).drop_duplicates(ignore_index=True) # Important: drop duplicates
df_demo_allsubjects = pd.read_csv(os.path.join(timb_task_sensor_path, "demographics_allSubjects.csv"))

# Get list of subjects from the demographics csv
subject_list = list(df_demo_allsubjects.loc[(df_demo_allsubjects['RawExists'] == 1)]['SubjectID'])

**Define timing parameters for selected events**

In [19]:
# Define time limits before and after the stimulus (i.e. time since the last BP and time until the next BP)
pre_trial_time = 1
post_trial_time = 15

# Define suffix for output tfr file
tfr_suffix = f'_epoch_tfrs_no_baseline_{pre_trial_time}s-pre_{post_trial_time}s-post_BPtrial-tfr.h5'

**Define function to do the work**

This will load the raw data, perform preprocessing, epoch according to our trial selection criteria, and compute and save TFRs to disk

In [20]:
# Define a function to load the data, subset long trials, and compute TFRs
def compute_long_bp_tfrs(subject): 

    # Define input files
    raw_fif_path = os.path.join(raw_path, subject, 'task', raw_fif_fname)
    events_path = os.path.join(event_numpy_path, subject + '-eve.txt')
    ica_path = os.path.join(timb_task_sensor_path, 'TaskSensorAnalysis_transdef', subject, ica_fname)

    # Define output files    
    out_path = os.path.join(data_path, 'proc_data', subject)
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    tfr_path = os.path.join(out_path, f'{subject}_{tfr_suffix}')  

    # Skip if the output file already exists
    if os.path.exists(tfr_path):

        print("Skipping subject " + subject + " because the output file already exists")

    #     # Some hd5 files have been corrupted, causing "OSError: Can't synchronously read data (inflate() failed)"
    #     # when trying to load them. We'll try to catch the error: if it occurs, delete the file and recompute it.
    #     try:
    #         tfr = mne.time_frequency.read_tfrs(tfr_path)[0]
    #         return # return if the file exists and does not cuase an error
    #     except OSError:
    #         print("Deleting corrupted file: " + tfr_path)
    #         os.remove(tfr_path)


    # If raw or events files do not exist, skip
    if not os.path.exists(raw_fif_path):
        print("Skipping subject " + subject + " because raw file does not exist")
        return
    if not os.path.exists(events_path):
        print("Skipping subject " + subject + " because events file does not exist")
        return

    # Pull the rows of df_demo_allsubjects and df_trial_timings_allsubjects for this subject
    df_trial_timings = df_trial_timings_allsubjects[df_trial_timings_allsubjects['subject'] == subject]

    # Subset rows of df_trial_timings to only include trials fitting our pre- and post-trial times
    df_trial_timings_subset = df_trial_timings[(df_trial_timings['t_since_prev_trial'] >= pre_trial_time) 
                                        & (df_trial_timings['t_until_next_trial'] >= post_trial_time)]

    # Convert this_trial_t to ms
    this_trial_t = df_trial_timings_subset['this_trial_t']*1000

    # Read in the events file for this subject    
    events = mne.read_events(events_path)

    # Select button press events (the most frequent trigger type)
    events_bp = events[events[:,2] == mode(events[:,2])] 

    # Find rows of events where onset time matches elements of this_trial_t
    events_bp_long = events_bp[np.isin([i[0] for i in events_bp], this_trial_t), :]

    # Pass if there are no events fitting our criteria
    if events_bp_long.size==0:
        print("Skipping subject " + subject + " because there are no events fitting our criteria")
        return

    # Read raw data
    raw = mne.io.read_raw_fif(raw_fif_path, preload=True)                                                                                      

    # Apply filtering
    raw_filt = raw.copy().filter(0, 40)

    # Epoch the raw data. Note that our epochs will be 1s wider (on either side) than pre/post_trial_time, to avoid edge effects 
    # in the final tfrs. We will crop the tfrs by 1 sec on either side later on
    epochs = mne.Epochs(raw_filt, events_bp_long, tmin=-(pre_trial_time+1), tmax=(post_trial_time+1), baseline=None) 

    # Load pre-computed ICA file from Tim's old TaskSensor folder.
    try:
        ica = mne.preprocessing.read_ica(ica_path)
    
    # If it does not exist, look for it in this subject's output directory
    except FileNotFoundError:
        print("ICA file not found in Tim's TaskSensorAnalysis folder. Looking in our output directory.")
        try:
            new_ica_path = os.path.join(out_path, os.path.basename(ica_path))
            ica = mne.preprocessing.read_ica(new_ica_path)
    
        # If the ica file does not exist in our output directory, compute ICA here
        except FileNotFoundError:

            print('!! Running ICA !!')
            reject = dict(grad=4000e-13, mag=5e-12)
            picks = mne.pick_types(raw.info, meg=True, eeg=False, eog=True,
                                stim=False, exclude='bads')
            
            ica = ICA(n_components=0.99, method='fastica')

            # Added by LB due to ica.fit() throwing a Runtime Error (No clean segment found) on subject CC510395
            try:  # Added by LB
                ica.fit(raw, picks=picks, reject=reject)  # indented by LB
            except:  # Added by LB
                print('## Failed to fit ICA ##')
                return # Added by LB

            n_max_ecg, n_max_eog = 3, 3

            # Reject bad EOG components following mne procedure
            try:
                eog_epochs = create_eog_epochs(raw, tmin=-0.5, tmax=0.5, reject=reject)
                eog_inds, scores = ica.find_bads_eog(eog_epochs)
                eog_inds = eog_inds[:n_max_eog]
                ica.exclude.extend(eog_inds)
            except:
                print("""Subject {0} had no eog/eeg channels""".format(str(subject)))

            # Reject bad ECG compoments following mne procedure
            ecg_epochs = create_ecg_epochs(raw, tmin=-0.5, tmax=0.5)
            ecg_inds, scores = ica.find_bads_ecg(ecg_epochs, method='ctps')
            ecg_inds = ecg_inds[:n_max_ecg]
            ica.exclude.extend(ecg_inds)   

            # Save ICA to this subject's output folder
            ica.save(new_ica_path)

    # Apply ica to the epochs
    epochs_icad = ica.apply(epochs.load_data())

    # Pick sensors of interest
    epochs_icad_picks = epochs_icad.copy().pick(['MEG0211', 'MEG1311'])

    # Set parameters for TFR
    freqs = np.arange(1, 40, 1) 
    n_cycles = freqs / 2.0

    # Compute TFR for each trial
    tfr = epochs_icad_picks.compute_tfr(method='morlet', freqs=freqs, n_cycles=n_cycles, average=False, decim=10) 

    # Crop time back to our desired window, which will eliminate edge effects
    tfr_cropped = tfr.copy().crop(tmin=-pre_trial_time, tmax=post_trial_time)

    # Save to disk
    tfr_cropped.save(tfr_path, overwrite=True)



**Do the work**

In [ ]:
# Loop through subjects and compute trial TFR(s) for each
for subject in tqdm(subject_list[:1]):
    compute_long_bp_tfrs(subject)

In [ ]:
print(subject_list[:1])